# 가격 예측을 위한 최종 부스팅 모델 선정
*수출입 데이터를 포함한 전체 데이터셋을 사용하여, 세 가지 주요 부스팅 모델(GradientBoosting, XGBoost, LightGBM)의 성능을 비교하고 최적의 하이퍼파라미터를 탐색합니다.*

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

## 1. 데이터 불러오기 및 통합 전처리

In [ ]:
# 데이터 로드
cold_df = pd.read_excel('../data//mon_cold.xlsx')
wind_df = pd.read_excel('../data//mon_wind.xlsx')
hot_df = pd.read_excel('../data//mon_hot.xlsx')
price_df = pd.read_excel('../data//region_price.xlsx')
weather_df = pd.read_csv('../data//region_weather.csv')
trade_df = pd.read_excel('../data//spinach_cucumber_df.xlsx')

# 데이터 전처리 (이전과 동일)
cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')
for df in [price_df, weather_df, trade_df]:
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce').dt.strftime('%Y-%m')
merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, trade_df, on=['품목', '날짜'], how='left')
merged_df.dropna(subset=['평균가격'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()
merged_df.fillna(0, inplace=True)

# 특성 공학
merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month
print('데이터 통합 및 전처리 완료.')

데이터 통합 및 전처리 완료.


## 2. 부스팅 모델 비교, 튜닝 및 평가

In [6]:
def find_best_boosting_model(df, vegetable_name):
    target_df = df[df['품목'] == vegetable_name].copy()
    target_df['지역'] = target_df['지역'].astype('category').cat.codes

    features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '지역', '연도', '월', '수출중량', '수입중량']
    target = '평균가격'

    X = target_df[features]
    y = target_df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 모델 및 하이퍼파라미터 그리드 정의
    models_and_params = [
        ('GradientBoosting', GradientBoostingRegressor(random_state=42), {'n_estimators': [100, 200], 'max_depth': [3, 5], 'learning_rate': [0.05, 0.1]}),
        ('XGBoost', XGBRegressor(random_state=42), {'n_estimators': [100, 200, 300], 'max_depth': [3, 5, 7], 'learning_rate': [0.05, 0.1]}),
        ('LightGBM', LGBMRegressor(random_state=42, verbose=-1), {'n_estimators': [100, 200, 300], 'max_depth': [5, 7], 'learning_rate': [0.05, 0.1], 'num_leaves': [31, 63]})
    ]

    results_summary = []

    print(f'--- {vegetable_name} 부스팅 모델 튜닝 및 평가 시작 ---')

    for name, model, params in models_and_params:
        print(f'{name} 모델 튜닝 중...')
        grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, n_jobs=-1, scoring='r2', verbose=0)
        grid_search.fit(X_train, y_train)
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test)

        # 평가 지표 계산
        mae = mean_absolute_error(y_test, y_pred)
        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_test, y_pred)

        results_summary.append({'Model': name, 'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R2': r2, 'Best Params': grid_search.best_params_})

    # 최종 결과 출력
    results_df = pd.DataFrame(results_summary).sort_values(by='R2', ascending=False).reset_index(drop=True)
    best_model_info = results_df.iloc[0]

    print(f'--- {vegetable_name} 최종 모델 평가 결과 ---')
    print(results_df.to_string())
    print(f'=> {vegetable_name}의 최종 최적 모델은 {best_model_info["Model"]} 입니다.')
    print(f'   - R2 Score: {best_model_info["R2"]:.4f}')
    print(f'   - Best Hyperparameters: {best_model_info["Best Params"]}')
    print('-'*80)

# 함수 실행
find_best_boosting_model(merged_df, '시금치')
find_best_boosting_model(merged_df, '오이')

--- 시금치 부스팅 모델 튜닝 및 평가 시작 ---
GradientBoosting 모델 튜닝 중...
XGBoost 모델 튜닝 중...
LightGBM 모델 튜닝 중...
--- 시금치 최종 모델 평가 결과 ---
              Model          MAE           MSE         RMSE        R2                                                                    Best Params
0  GradientBoosting  2816.530927  3.355529e+07  5792.692787  0.712014                   {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
1          LightGBM  2842.609523  3.409037e+07  5838.696287  0.707422  {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 31}
2           XGBoost  2839.077850  3.570659e+07  5975.499357  0.693551                    {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
=> 시금치의 최종 최적 모델은 GradientBoosting 입니다.
   - R2 Score: 0.7120
   - Best Hyperparameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200}
--------------------------------------------------------------------------------
--- 오이 부스팅 모델 튜닝 및 평가 시작 ---
GradientBoosting 모델 튜닝 